In [1]:
import os
import joblib
import sqlite3
import numpy as np
import pandas as pd
import time
import json
import faiss
from tqdm import tqdm
from sklearn.metrics import average_precision_score
from sentence_transformers import SentenceTransformer
from TextPreprocessor import TextPreprocessor

C:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bayan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# --- الإعدادات ---
SOURCE = "quora"
TOP_N = 10
MODEL_NAME = "all-MiniLM-L6-v2"
INDEX_DIR = "indexes"
MODEL_DIR = "models"
DB_PATH = "ir_project.db"

In [3]:
# --- تحميل FAISS Index ---
index_path = os.path.join(INDEX_DIR, f"faiss_index_{SOURCE}_bert.index")
print(f"📥 تحميل FAISS index من: {index_path}")
index = faiss.read_index(index_path)

📥 تحميل FAISS index من: indexes\faiss_index_quora_bert.index


In [4]:
# --- تحميل معرّفات الوثائق ---
doc_ids = joblib.load(os.path.join(MODEL_DIR, f"bert_{SOURCE}_doc_ids.joblib"))

In [5]:
# --- تحميل النموذج والمعالجة ---
model = SentenceTransformer(MODEL_NAME)
preprocessor = TextPreprocessor()

In [6]:
# --- ربط قاعدة البيانات ---
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [7]:
cursor.execute("SELECT query_id, query_text FROM queries WHERE source = ?", (SOURCE,))
queries = cursor.fetchall()

In [8]:
cursor.execute("SELECT query_id, doc_id FROM qrels WHERE source = ?", (SOURCE,))
qrels_raw = cursor.fetchall()

In [9]:
qrels = {}
for qid, doc_id in qrels_raw:
    qrels.setdefault(qid, set()).add(doc_id)

In [10]:
# --- التقييم ---
precisions, recalls, average_precisions, reciprocal_ranks = [], [], [], []

In [11]:
print(f"\n⚙️ بدء التقييم على {len(queries)} استعلام...\n")
start_time = time.perf_counter()


⚙️ بدء التقييم على 5000 استعلام...



In [12]:
for qid, qtext in tqdm(queries):
    if qid not in qrels:
        continue

    relevant_docs = qrels[qid]
    tokens = preprocessor.preprocess(qtext)
    if not tokens:
        continue

    query_vec = model.encode([" ".join(tokens)]).astype("float32")

    # --- البحث باستخدام FAISS ---
    D, I = index.search(query_vec, TOP_N)
    top_docs = [doc_ids[i] for i in I[0] if i < len(doc_ids)]
    scores = D[0]

    # Precision@10
    hits = [1 if doc in relevant_docs else 0 for doc in top_docs]
    precisions.append(sum(hits) / TOP_N)
    recalls.append(sum(hits) / len(relevant_docs))

    # MAP
    y_true = [1 if doc in relevant_docs else 0 for doc in top_docs]
    y_scores = [1.0 / (i + 1) for i in range(len(y_true))]  # أو استخدم scores مباشرة إن أردت
    try:
        ap = average_precision_score(y_true, y_scores)
    except:
        ap = 0.0
    average_precisions.append(ap)

    # MRR
    for rank, doc in enumerate(top_docs, 1):
        if doc in relevant_docs:
            reciprocal_ranks.append(1 / rank)
            break
    else:
        reciprocal_ranks.append(0.0)

  0%|          | 0/5000 [00:00<?, ?it/s]C:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  0%|          | 1/5000 [00:00<43:41,  1.91it/s]C:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  0%|          | 9/5000 [00:01<08:12, 10.13it/s]C:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  0%|          | 13/5000 [00:01<07:54, 10.50it/s]C:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
  0%|          | 15/5000 [00:01<08:51,  9.38it/s]C:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No p

In [13]:
end_time = time.perf_counter()
elapsed = round(end_time - start_time, 2)

In [14]:
# --- النتائج ---
results = {
    "Precision@10": round(np.mean(precisions), 4),
    "Recall": round(np.mean(recalls), 4),
    "MAP": round(np.mean(average_precisions), 4),
    "MRR": round(np.mean(reciprocal_ranks), 4),
    "Execution Time (seconds)": elapsed,
    "Queries Evaluated": len(precisions)
}

In [15]:
# --- عرض النتائج ---
pd.DataFrame([results])

,Precision@10,Recall,MAP,MRR,Execution Time (seconds),Queries Evaluated
0,0.1163,0.8594,0.7398,0.7539,427.02,4999


In [16]:
# --- حفظ النتائج ---
with open(f"faiss_bert_evaluation_{SOURCE}.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

In [17]:
conn.close()